# Travaux Pratiques 3

Dans ce TP, vous allez manipuler des méthodes d'estimation basées sur la vraisemblance.

En présence de valeurs manquantes, on maximise la vraisemblance observée :
$$\theta^{ML} \in \textrm{argmax}_\theta \: L_{{\mathrm{{obs}}}}(\theta;X_{\mathrm{obs}(M)}),$$

$$\mathrm{avec} \: L_{{\mathrm{{obs}}}}(\theta;X_{\mathrm{obs}(M)})={\int} p(X;\theta) {dX_{\mathrm{mis}(M)}}.$$

Cela permet d'estimer le paramètre $\theta$ de la distribution des données $p(X; \theta)$ et possiblement de prédire les valeurs manquantes.


L'exercice 1 vous permettra de découvrir l'algorithme Espérance-Maximisation dans un cas bivarié Gaussien pour estimer les paramètres en présence de valeurs manquantes. Dans l'exercice 2, vous verrez pourquoi et comment ces méthodes peuvent aussi être utilisées pour faire de l'imputation. Dans l'exercice 3, vous manipulerez les méthodes de rang faible, et dans l'exercice 4, une méthode utilisant l'apprentissage profond.

# Importation de librairies

In [ ]:
### Classical libraries
import numpy as np
import pandas as pd

### Specific for missing values
from sklearn.impute import SimpleImputer

### Dataset
from sklearn.datasets import load_breast_cancer

### Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors

### Pytorch for exercice 4 (deep learning)
import torch
import torchvision
import torch.nn as nn
import torch.distributions as td
from torch import optim

# Exercice 1: algorithme Espérance-Maximisation

Considérons un échantillon bivarié (*i.e.* à $d=2$ variables) gaussien,

$$
\left( X_{i.} \right)_{1\leq i\leq n}
=\left( X_{i0}, X_{i1} \right)_{1\leq i\leq n}
$$

de taille $n$, de moyenne $\mu$ et de matrice de covariance $\Sigma$, en notant :

$$
\mu = \begin{bmatrix}
  \mu_0 \\[6pt] \mu_1
\end{bmatrix}, \quad
\Sigma = \begin{bmatrix}
  \sigma_{0} & \sigma_{01} \\[6pt]
  \sigma_{01} & \sigma_{1}
\end{bmatrix},
$$

Supposons également qu'il y a des valeurs manquantes de type Missing Completely At Random (MCAR) sur $X_{.1}$: le manque des données ne dépend pas des valeurs des données elles-mêmes et le mécanisme de données manquantes peut donc être ignoré dans l'analyse statistique.

In [ ]:
np.random.seed(0)

n = 100
d = 2
mu0 = 5.
mu1 = 1.
sig0 = 1.
sig1 = 1.
sig01 = 0.5

mean = np.array([mu0, mu1])
cov = np.array([
    [sig0, sig01],
    [sig01, sig1]
    ])

xfull = np.random.multivariate_normal(mean, cov, size=n)

In [ ]:
p = 0.4

xmiss = np.copy(xfull)
miss_id = (np.random.uniform(0, 1, size=n) < p)
xmiss[miss_id, 1] = np.nan

M = np.isnan(xmiss)

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MCAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

## Question 1 : estimateur du maximum de vraisemblance

La log-vraisemblance observée à maximiser est
$$\ell_{\textrm{obs}}(\theta;X_{.0},X_{\mathrm{obs}(M_{.1})1})=\sum_{i=1}^n \int \log p(X_{i.};\theta) dX_{\mathrm{mis}(M_{i1})1},$$
où $\theta=(\mu,\Sigma)$. Par convention, on a $X_{i1}=X_{\mathrm{mis}(M_{i1})1}$ si $M_{i1}=1$, c'est-à-dire lorsque $X_{i1}$ est manquant. De même, on a $X_{i1}=X_{\mathrm{obs}(M_{i1})1}$ si $M_{i1}=0$, c'est-à-dire lorsque $X_{i1}$ est observé.
Dans la suite, vous pouvez considérer, sans perdre de généralité, que les $r$ premières observations sont observées et les $n-r$ restantes sont manquantes, c'est-à-dire $M_{i1}=0$ pour $i=1,\dots,r$ et $M_{i1}=1$ pour $i=r+1,\dots,n$.

Vous allez montrer dans la question 1a que son expression est, à des constantes près:
$$\ell(\mu,\Sigma;X_{.0},X_{\mathrm{obs}(M_{.1})1})=-\frac{n}{2}\log(\sigma_{0}^2)-\frac{1}{2}\sum_{i=1}^{n}\frac{(X_{i0}-\mu_0)^2}{\sigma_{0}^2}
-\frac{r}{2}\log\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2 \\
-\frac{1}{2}\sum_{i=1}^{r}\frac{(X_{i1}-\mu_1-\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0))^2}{\left(\sigma_{11}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2}$$
et dans la question 1b, vous allez donner l'expression du maximum de vraisemblance. La question 1c est de l'implémentation.  

### Question 1a

Voici les premières étapes pour obtenir la vraisemblance observée.

\begin{align}
\ell_{\textrm{obs}}(\theta;X_{.0},X_{\mathrm{obs}(M_{.1})1})&=\sum_{i=1}^n \int \log p(X_{i.};\theta) dX_{\mathrm{mis}(M_{.1})1} \\
&=\sum_{i=1}^n \int \log p(X_{i0})p(X_{i0}|X_{i1};\theta) dX_{\mathrm{mis}(M_{.1})1} \\
&= \sum_{i=1}^n \log p(X_{i0}) + \sum_{i=1}^n \int \log p(X_{i0}|X_{i1};\theta) dX_{\mathrm{mis}(M_{.1})1}
\end{align}

Détaillez les deux termes, en utilisant pour le second les formules classiques d'un vecteur gaussien rappelées ci-dessous:

$X_{i1}|X_{i0} \sim N(\mathbb{E}[X_{i1}|X_{i0}],\mathrm{Var}(X_{i1}|X_{i0}))$ avec
\begin{align*}
\mathbb{E}[X_{i1}|X_{i0}]&=\mu_1+\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0) \\
\mathrm{Var}(X_{i1}|X_{i0})&=\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}
\end{align*}

### Solution

Pour le premier terme, comme $X_{.0}\sim N(\mu_0,\sigma_{0})$, on a simplement
$\sum_{i=1}^n \log p(X_{i0};\theta)= -\sum_{i=1}^n \frac{(X_{i0}-\mu_0)^2}{2\sigma_{0}^2}-\frac{n}{2}\log\sigma_{0}^2$.

Pour le second terme, il y a deux cas. Si $X_{i1}$ est manquant, l'intégrale vaut 1 (densité intégrée).
Si $X_{i1}$ est observé (càd pour $X_{i1}$ avec $i=1,\dots,r$), le terme $p(X_{i1}|X_{i1};\theta)$ sort de l'intégrale et on utilise les formules classiques pour obtenir $-\frac{r}{2}\log\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2
-\frac{1}{2}\sum_{i=1}^{r}\frac{(X_{i1}-\mu_1-\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0))^2}{\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2}$.



### Question 1b

L'estimateur du maximum de vraisemblance est
$\theta^{ML} \in \textrm{argmin}_\theta \: \ell_{\textrm{obs}}(\theta;X_{.0},X_{\mathrm{obs}(M_{.1})1})$. Donnez son expression pour estimer la moyenne, en considérant $\Sigma$ connu pour simplifier les calculs.

### Solution

À $\Sigma$ fixé (connu), on dérive la vraisemblance observée par rapport à $\mu$, les expressions suivantes sont obtenues:

\begin{align*}
    \nabla_{\mu_0} \ell(\mu,\Sigma;X_{.0},X_{.1})&=\sum_{i=1}^{n}\frac{X_{i0}-\mu_0}{\sigma_{0}^2}
 \\
 \nabla_{\mu_1}\ell(\mu,\Sigma;X_{.0},X_{.1})&=\sum_{i=1}^{r}\frac{X_{i1}-\mu_1-\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0)}{\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2}
\end{align*}

\begin{align*}
\nabla_{\mu_0} \ell(\mu,\Sigma;X_{.0},X_{.1})=0 &\Longleftrightarrow {\mu}^{ML}_0 = \frac{1}{n}\sum_{i=1}^n X_{i0} \\
\nabla_{\mu_1} \ell(\mu,\Sigma;X_{.0},X_{.1})=0 &\Longleftrightarrow {\mu}^{ML}_1 = \frac{1}{r}\sum_{i=1}^r X_{i1} + \frac{\sigma_{01}}{\sigma_{0}}\left({\mu}_0^{ML}-\frac{1}{r} \sum_{i=1}^r X_{i0}\right),
\end{align*}
où on remplace $\mu_0$ par son estimateur ${\mu}^{ML}_0$ dans l'expression de ${\mu}^{ML}_1$ (méthode de plug-in). On ne fait pas la preuve de concavité pour la vraisemblance $\ell$.

### Question 1c

Implémentez l'estimateur du maximum de vraisemblance pour la moyenne à partir de l'expression trouvée en question 1b.

### Solution

In [ ]:
mu0_ML = np.mean(xmiss[:, 0])
mu1_ML = np.mean(xmiss[~miss_id, 1]) + sig01 / sig0 * (mu0_ML - np.mean(xmiss[~miss_id, 0]))

In [ ]:
print("Estimateur de mu0 :", mu0_ML)
print("Estimateur de mu1 :", mu1_ML)

## Question 2 : Etape E

Comme vous avez vu dans la vidéo du module 3, le paramètre $\theta$ peut aussi être estimé de manière itérative avec l'algorithme EM.

Il y a une étape d'initialisation où l'on obtient $\theta^{(0)}$, puis deux étapes répétées jusqu'à convergence :
* l'étape E, où l'espérance conditionnelle suivante est calculée (itération $t$):
$$Q(\theta;{\theta}^{(t)})=\mathbb{E}[\quad \ell(\theta;X)\quad\big|X_{.0},X_{\mathrm{obs}(M_{.1})1},{\theta}^{(t)}],$$
avec $\ell(\theta;X)=\sum_{i=1}^n\log p(X;\theta)$ la log-vraisemblance complète.

* l'étape M, où l'espérance conditionnelle est maximisée pour mettre à jour les paramètres
$${\theta}^{(t+1)}\in\textrm{argmax}_\theta \: Q(\theta;{\theta}^{(t)})$$

Vous allez détailler l'étape E.



La log-vraisemblance complète est donnée ci-dessous:

\begin{align*}
\ell(\theta;X)&=-\frac{n}{2}\log(\mathrm{det}(\Sigma))-\frac{1}{2}\sum_{i=1}^n\begin{pmatrix} X_{i0}-\mu_0 & X_{i1}-\mu_1
\end{pmatrix}\Sigma^{-1}\begin{pmatrix} X_{i0}-\mu_0 & X_{i1}-\mu_1
\end{pmatrix}^T \\
&= \frac{n}{2}\log(\mathrm{det}(\Sigma))-\frac{1}{2}\sum_{i=1}^n (X_{i0}-\mu_0)^2\tilde{\sigma}_{0} + 2(X_{i0}-\mu_0)(X_{i1}-\mu_1)(\tilde{\sigma}_{01})^2 + (X_{i1}-\mu_1)^2\tilde{\sigma}_{1},
\end{align*}
avec $\Sigma^{-1}=\begin{pmatrix} \tilde{\sigma}_{0} & \tilde{\sigma}_{01} \\
\tilde{\sigma}_{01} & \tilde{\sigma}_{1}  
\end{pmatrix}$.

$\ell(\theta;X)$ est linéaire en les quantités suivantes: $\sum_{i=1}^n X_{i0}$, $\sum_{i=1}^n X_{i0}^2$, $\sum_{i=1}^n X_{i1}$, $\sum_{i=1}^n X_{i1}^2$ et $\sum_{i=1}^n X_{i0}X_{i1}$.
Pour obtenir l'expression de l'espérance conditionnelle, il suffit donc de calculer les quantités suivantes:
\begin{align*}
s_j&=\mathbb{E}\left[ \sum_{i=1}^n X_{ij}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right], j=0,1 \\
s_{jj}&=\mathbb{E}\left[ \sum_{i=1}^n X_{ij}^2|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right], j=0,1 \\
s_{jk}&=\mathbb{E}\left[ \sum_{i=1}^n X_{ij}X_{ik}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right], j=0, k=1
\end{align*}


### Question 2a

Calculez $s_0$ et $s_{00}$.

### Solution

Les expressions pour $s_0$ et $s_{00}$ sont les plus faciles à obtenir. En effet, on conditionne par rapport à $X_{.0}$, on obtient donc :
$$s_0 = \sum_{i=1}^n X_{i0}, \quad  s_{00} = \sum_{i=1}^n X_{i0}^2.$$

### Question 2b

Calculez $s_1$. La technique est la même que pour le calcul du second terme de la question 1a.

### Solution

\begin{align*}
        \mathbb{E}\left[ X_{i1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]&=  \int X_{i1} p(X_{\textrm{mis}(M_{i1})1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)})dX_{\textrm{mis}(M_{i1})1} \\
        &=\begin{cases}
		X_{\textrm{obs}(M_{i1})1}  &\textrm{ si } X_{i1} \textrm{ est observé} \\
		\int X_{\textrm{mis}(M_{i1})1} p(X_{\textrm{mis}(M_{i1})1}|X_{i0},{\theta}^{(t)})dX_{\textrm{mis}(M_{i1})1} &\textrm{ sinon.}
	\end{cases}
\end{align*}
Dans le premier cas, si $X_{i1}$ est observé, $X_{i1}=X_{\textrm{obs}(M_{i1})1}$ et peut être sorti de l'intégrale, qui vaut $1$. Dans le second cas, c'est simplement l'espérance de la distribution conditionnele de $X_{i1}$ sachant $X_{i0}$. Pour cela, on peut utiliser les formules classiques des vecteurs gaussiens (rappelées en question 1a) et obtenir :
\begin{align*}
        \mathbb{E}\left[ X_{i1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]&= \begin{cases}
		X_{i1} &\textrm{ si } X_{i1} \textrm{ est observé} \\
		\mu_1^{(t)} + \frac{\sigma_{01}^{(t)}}{\sigma_{0}^{(t)}}( X_{i0}-\mu_0^{(t)}) &\textrm{ sinon.}
	\end{cases}
\end{align*}

Pour les termes $s_{11}$ et $s_{01}$, on emploie exactement la même stratégie que dans la question 2b pour obtenir:
\begin{align}
s_{11}&=\mathbb{E}\left[ \sum_{i=1}^nX_{i1}^2|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]=\sum_{i=1}^{r} X_{i1}^2 + \sum_{i=r+1}^n \left(\left(\mu_1^{(t)} + \frac{\sigma^{(t)}_{01}}{\sigma^{(t)}_{0}}( X_{i0}-\mu^{(t)}_0)\right)^2+\sigma^{(t)}_{1}-\frac{(\sigma^{(t)}_{01})^2}{\sigma^{(t)}_{0}}\right) \\
s_{01}&=\mathbb{E}\left[ \sum_{i=1}^n X_{i0}X_{i1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]=\sum_{i=1}^{r} X_{i0}X_{i1} + \sum_{i=r+1}^n X_{i0}\left(\mu^{(t)}_1 +\frac{\sigma^{(t)}_{01}}{\sigma^{(t)}_{0}}\left( X_{i0}-\mu^{(t)}_0\right)\right)
\end{align}

## Question 3: Etape M

Sans détailler les calculs, comment pouvez-vous maximiser la vraisemblance à l'étape M ?

### Solution

On retrouve l'estimateur classique du maximum de vraisemblance dans le cas Gaussien (voir par exemple section 2.3.4 du
[livre de Bishop, 2006](https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf)), en remplaçant les quantités impliquant des valeurs manquantes par les espérances conditionnelles calculées dans la question 2.

\begin{align*}
    \mu_0^{(t+1)}&=\frac{s_0}{n} \\
    \mu_1^{(t+1)}&=\frac{s_1}{n} \\
    \sigma_{0}^{(t+1)}&=\frac{s_{00}}{n}-(\mu_0^{(t+1)})^2 \\
    \sigma_{1}^{(t+1)}&=\frac{s_{11}}{n}-(\mu_1^{(t+1)})^2 \\
    \sigma_{01}^{(t+1)}&=\frac{s_{01}}{n}-(\mu_0^{(t+1)}\mu_1^{(t+1)})
\end{align*}

## Question 4: Mise en pratique

### Question 4a

Il faut d'abord trouver une initialisation pour $\theta^{(0)}$. Vous allez initialiser l'algorithme avec les quantités empiriques (moyenne et matrice de variance-covariance) calculées sur les lignes complètes. Ce n'est pas toujours une solution viable (par exemple, lorsqu'il y a trop de valeurs manquantes), une autre solution peut être d'imputer les valeurs manquantes par une méthode pas trop coûteuse et de calculer les quantités empiriques pour $\theta^{(0)}$.

Ecrivez le code qui permet l'initialisation. Que remarquez-vous ?

### Solution

Les résultats pour la matrice de variance-covariance $\Sigma$ sont biaisés.

In [ ]:
mu_init = np.nanmean(xmiss,axis=0)
mu_init

In [ ]:
Sigma_init = np.array(pd.DataFrame(xmiss).cov())
Sigma_init

### Question 4b

Ecrivez deux fonctions pour l'étape E et M de l'algorithme.

In [ ]:
def Estep(xmiss, mu, Sigma, miss_id):

    ### TO COMPLETE ###

    return {
        's0': np.sum(s0),
        's1': np.sum(s1),
        's00': np.sum(s00),
        's11': np.sum(s11),
        's01': np.sum(s01)
    }

In [ ]:
def Mstep(xmiss, s0, s1, s00, s11, s01):

    ### TO COMPLETE ###

    return {'mu': mu, 'Sigma': Sigma}

### Solution

In [ ]:
def Estep(xmiss, mu, Sigma, miss_id):

    n = xmiss.shape[0]

    # For the variable X0
    s0 = xmiss[:, 0]
    s00 = xmiss[:, 0] ** 2

    s1 = np.zeros(n)
    s11 = np.zeros(n)

    # For observed values of X1
    s1[~miss_id] = xmiss[~miss_id, 1]
    s11[~miss_id] = xmiss[~miss_id, 1] ** 2

    # For missing values of X1
    s1[miss_id] = mu[1] + (Sigma[0,1] / Sigma[0,0]) * (xmiss[miss_id, 0] - mu[0])
    s11[miss_id] = (
        s1[miss_id] ** 2 +
        Sigma[1,1] - (Sigma[0,1] ** 2) / Sigma[0,0]
    )

    s01 = s0 * s1

    return {
        's0': np.sum(s0),
        's1': np.sum(s1),
        's00': np.sum(s00),
        's11': np.sum(s11),
        's01': np.sum(s01)
    }

In [ ]:
def Mstep(xmiss, s0, s1, s00, s11, s01):

    n = xmiss.shape[0]

    mu0 = s0 / n
    mu1 = s1 / n

    sig0 = s00 / n - mu0 ** 2
    sig1 = s11 / n - mu1 ** 2
    sig01 = s01 / n - mu0 * mu1

    mu = np.array([mu0, mu1])
    Sigma = np.array([
        [sig0, sig01],
        [sig01, sig1]
    ])

    return {'mu': mu, 'Sigma': Sigma}

### Question 4c

Appliquez l'algorithme sur 50 itérations.

### Solution

In [ ]:
mu_EM = mu_init
Sigma_EM = Sigma_init

for i in range(50):

  # E-step
  res_Estep = Estep(xmiss, mu_EM, Sigma_EM, miss_id)
  s0 = res_Estep["s0"]
  s1 = res_Estep["s1"]
  s00 = res_Estep["s00"]
  s11 = res_Estep["s11"]
  s01 = res_Estep["s01"]

  # M-step
  res_Mstep = Mstep(xmiss, s0, s1, s00, s11, s01)
  mu_EM = res_Mstep["mu"]
  Sigma_EM = res_Mstep["Sigma"]


In [ ]:
mu_EM

In [ ]:
Sigma_EM

# Exercice 2: estimer pour imputer

Dans ce court exercice, vous allez imputer les valeurs manquantes à partir de l'estimation du paramètre $\theta$ de la distribution des données $p(X)$.

### Question 1

Dans le cas de l'exercice 1 (bivarié gaussien), proposez une stratégie pour imputer, et donc tirer un échantillon suivant la loi $p(X_{i1}|X_{i0})$.

### Solution

Dans le cas Gaussien bivarié, cette loi est explicite. On peut de nouveau utiliser les formules classiques d'un vecteur Gaussien rappelées en question 1a.

On a:
$X_{i1}|X_{i0} \sim N(\mathbb{E}[X_{i1}|X_{i0}],\mathrm{Var}(X_{i1}|X_{i0}))$ avec
\begin{align*}
\mathbb{E}[X_{i1}|X_{i0}]&=\mu_1+\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0) \\
\mathrm{Var}(X_{i1}|X_{i0})&=\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}
\end{align*}

## Question 2

Implémentez la stratégie choisie.

### Solution

In [ ]:
def imputation_EM(mu, Sigma, xmiss, miss_id):

  ximp = np.copy(xmiss)

  for i in range(xmiss.shape[0]):
    if miss_id[i]:
      mean = mu[1] + (Sigma[0, 1] / Sigma[0, 0]) * (xmiss[i, 0] - mu[0])
      cov = Sigma[1, 1] - (Sigma[0, 1] ** 2) / Sigma[0,0]
      ximp[i, 1] = np.random.normal(mean, cov, size=1)

  return ximp

In [ ]:
ximp_EM = imputation_EM(mu_EM, Sigma_EM, xmiss, miss_id)

In [ ]:
ax = sns.scatterplot(x=ximp_EM[:, 0], y=ximp_EM[:, 1], hue=M[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MCAR values imputed with the EM algorithm')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Imputed'], loc='lower right', fontsize='13')
;

# Exercice 3: méthodes de rang faible

Dans cet exercice, vous allez implémenter l'algorithme présenté dans la vidéo du module qui propose de prédire les valeurs manquantes avec une Analyse en Composantes Principales (ACP) itérative. En question 2, vous verrez plus de détails sur une de ses variantes, qui gère mieux le bruit dans les données, appelée `missMDA`: cette méthode est implémentée dans un package `R` très utilisé en pratique. En question 3, vous implémenterez vous-mêmes une autre de ses variantes, l'algorithme `softImpute`.

Dans les méthodes à rang faible, on fait l'hypothèse que la matrice de données peut être décomposée en une matrice à rang faible et un bruit gaussien.
$$X = \Theta + \epsilon,
$$
avec $\epsilon\sim N(0_d,\sigma^2I_{d\times d})$.

Pour prédire les valeurs manquantes dans $X$, on va estimer $\Theta$ en utilisant l'algorithme d'ACP itérative (présenté dans la vidéo). L'algorithme prend en entrée $r$, le nombre de dimensions à garder pour l'ACP, ainsi que le jeu de données incomplet $X^\textrm{NA}$.

- Il y a une étape d'**initialisation** d'imputation naïve (par exemple, par la moyenne ou même l'imputation par 0). On obtient alors un jeu de donnée complet $X^{(0)}$.

Deux étapes sont ensuites effectuées de manière itérative.

- l'**étape d'estimation**: on estime $\theta$ avec la décomposition en valeurs singulières pour laquelle on ne garde que les $r$ premières dimensions.
$$\Theta^{(t)}=\text{SVD}_{\textbf{r}}({X}^{(t)})=U_{{\textbf{r}}}D_{{\textbf{r}}}V_{{\textbf{r}}}^t,$$

- l'**étape d'imputation**: les valeurs manquantes sont prédites par la valeur de $\Theta^{(t)}$.

$$X^{(t+1)}=X \odot (\mathbf{1}_{n\times d}-{M}) + \: \Theta^{(t)} \odot M.$$


Dans cet exercice, on va considérer le même jeu de données réel complet *Breast Cancer Wisconsin* que dans les modules précédents (exercice 4 du module 1 et exercice 3 du module 2). On introduit 30% de valeurs manquantes de type MCAR.

In [ ]:
data = load_breast_cancer()
xfull = data['data'] # covariates, without missing values
diagnosis = data['target']   # target variable to predict, when the learning task is prediction
features_names = data['feature_names']

In [ ]:
pd.DataFrame(xfull, columns=features_names).head()

In [ ]:
n, d = xfull.shape

In [ ]:
np.random.seed(123)

p = 0.3
xmiss = np.copy(xfull)
for j in range(d):
  miss_id = np.random.uniform(0, 1, size=n) < p
  xmiss[miss_id, j] = np.nan
mask = np.isnan(xmiss)

Vous allez vous concentrer sur la tâche d'imputation et vous calculerez donc les erreurs quadratiques moyennes (MSE) pour évaluer les méthodes (score introduit dans le module 2).

In [ ]:
def mse(x_imp, x_true):
  n = len(x_true)
  return (1 / n) * np.sum((x_imp - x_true) ** 2)

## Question 1: ACP itérative

### Question 1a

Proposez une imputation naïve par la moyenne et une autre par 0 dans des fonctions que vous appelez `init_mean_imputation` et `init_zero_imputation`.

### Solution

In [ ]:
def init_mean_imputation(xmiss, mask):
    mean_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    ximp = mean_imputer.fit_transform(xmiss)
    return ximp

In [ ]:
def init_zero_imputation(xmiss, mask):
    ximp = xmiss.copy()
    ximp[mask] = 0
    return ximp

In [ ]:
pd.DataFrame(init_mean_imputation(xmiss, mask)).head()

In [ ]:
pd.DataFrame(init_zero_imputation(xmiss, mask)).head()

### Question 1b

Implémentez l'algorithme de l'ACP itérative dans la fonction suivante `iterative_ACP` qui prend comme arguments:
* `xmiss`: le jeu de données manquant,
* `r`: le nombre de dimensions à garder pour l'ACP,
* `maxit`: le nombre maximum d'itérations de l'algorithme. Un critère de convergence sera défini en question 3.

Vous utiliserez l'imputation par la moyenne comme initialisation.

Pour la décomposition en valeurs singulières, vous pouvez utilisez la fonction `linalg.svd` de `numpy` (documentation disponible [ici](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)).

In [ ]:
def iterative_ACP(xmiss, r, maxit):

    ximp = ...

    return ximp

### Solution

In [ ]:
def iterative_ACP(xmiss, r, maxit):

    mask = np.isnan(xmiss)

    ximp = init_mean_imputation(xmiss, mask)

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_r = d[:r]
        U_r = U[:, :r]
        V_r = V[:r, :]
        D_r = np.diag(d_r)
        svd_r = np.dot(U_r, np.dot(D_r, V_r))
        ximp[mask] = svd_r[mask]

    return ximp

In [ ]:
pd.DataFrame(iterative_ACP(xmiss, 5, 50)).head()

### Question 1c

Calculez la MSE, et comparez-la avec celle obtenue en utilisation une imputation par la moyenne.

### Solution

In [ ]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
ximp_mean = mean_imputer.fit_transform(xmiss)
mse_mean = mse(ximp_mean,xfull)
print("MSE imputation par la moyenne:", mse_mean)

In [ ]:
ximp_ACP = iterative_ACP(xmiss, 5, 1000)
mse_ACP = mse(ximp_ACP, xfull)
print("MSE ACP itérative:", mse_ACP)

On obtient une MSE légèrement plus faible que celle obtenue en imputant par la moyenne. On peut se demander si le nombre de dimension $r$ à garder dans l'ACP a été bien choisi ici.

### Question 1d

Pour choisir le nombre optimal $r$ de dimensions à garder pour l'ACP, une solution est de parcourir une grille de valeurs possibles et choisir le $r_\mathrm{opt}$ qui minimise la MSE sur cette grille.

Implémentez cette stratégie en utilisant la fonction suivante `additional_na`. Elle permet d'ajouter des valeurs manquantes de type MCAR de telle sorte à ce qu'il y ait au moins encore une valeur observée dans chaque ligne.

In [ ]:
def additional_na(xmiss, mask):

    xmiss_addna = np.copy(xmiss)

    for i in range(xmiss.shape[0]):
        idx_obs = np.argwhere(mask[i, :] == 0).reshape((-1))
        var_obs = np.random.choice(idx_obs) # choice of the observed variable
        idx_candidates = idx_obs[idx_obs != var_obs]
        var_miss = np.random.uniform(0, 1, size=len(idx_candidates)) < 0.5
        missing_idx = idx_candidates[var_miss]
        xmiss_addna[i, missing_idx] = np.nan
    mask_addna = np.isnan(xmiss_addna)

    return(xmiss_addna, mask_addna)

### Solution

In [ ]:
def choose_rank(xmiss, grid_r):

    mask = np.isnan(xmiss)

    list_error = []
    for r in grid_r:
        xmiss_addna, mask_addna = additional_na(xmiss, mask)
        ximp_ACP = iterative_ACP(xmiss_addna, r, 1000)

        list_error.append(np.sqrt(np.nanmean((ximp_ACP.flatten() - xmiss.flatten())**2)))

    return list_error

In [ ]:
grid_r = list(range(1, d))
res_rank = choose_rank(xmiss, grid_r)

In [ ]:
r_opt = grid_r[np.argmin(res_rank)]
ximp_ACP = iterative_ACP(xmiss, r_opt, 1000)
mse_ACP = mse(ximp_ACP, xfull)
print("MSE ACP itérative:", mse_ACP)
print("r optimal:", r_opt)

Le $r$ optimal choisi est 1. On utilise donc une projection des données sur une seule dimension pour pouvoir prédire les valeurs manquantes. On perd beaucoup d'information et une piste serait de mieux traiter le bruit contenu dans les données.

## Question 2 : ACP itérative régularisée

Dans l'algorithme d'ACP itérative, les valeurs manquantes sont remplacées par : $$\sum_{k=1}^{r}\sigma_k(X)u_{ik}v_{jk},$$
pour $i=1,\dots,n$ et $j=1,\dots,p$.

Pour régulariser, l'algorithme `missMDA` propose de remplacer les valeurs manquantes par :   $$\sum_{k=1}^{r}\left(\sigma_k(X)-\frac{{\sigma}^2}{\sigma_k(X)}\right)u_{ik}v_{jk}.$$

Cette méthode est implémentée en langage `R` dans le package `missMDA`. Des exemples d'utilisation sont disponibles sur ce [lien](http://factominer.free.fr/missMDA/index.html) et l'article associé est disponible [ici](https://www.jstatsoft.org/article/view/v070i01).

Faites une liste des hyperparamètres à choisir dans la méthode `missMDA`.

### Solution

Les hyperparamètres sont $r$ (le nombre de dimensions à garder dans l'ACP) et $\sigma^2$ (la variance du bruit supposé gaussien).

Pour estimer $r$, on peut utiliser une méthode semblable à celle choisie dans la question 1. Pour estimer $\sigma^2$, il a été proposé d'utiliser la somme des carrés des résidus divisé par $nd-\#\textrm{param}$, où $n$ et $d$ sont les dimensions de la matrice de données et $\#\textrm{param}=nr+pr-r^2$, le nombe de paramètres à estimer (ici nombre de paramètres dans une décomposition en valeurs singulières). Plus de détails sont disponibles [ici](https://arxiv.org/pdf/1602.01206).

Notons qu'il y a une fonction `estim_ncpPCA` du package `missMDA`qui propose une validation croisée pour estimer le nombre de dimensions à garder dans l'ACP.

## Question 3: ACP itérative à seuil doux

Une autre méthode qui permet de traiter le bruit dans les données est d'utiliser un seuillage doux avec l'algorithme `softImpute`. Tout comme `missMDA`, l'algorithme permet de mieux tenir compte du bruit contenu dans les données que l'ACP itérative classique, en remplaçant les valeurs manquantes par:
$$\sum_{k=1}^{d}\max((\sigma_k(X)-\lambda),0)u_{ik}v_{jk},$$
avec $\lambda>0$.

L'estimation de $\theta$ est régularisée dans le sens où le problème d'optimisation devient ici:

$$\theta \in \textrm{argmin}_\theta \| (X - \Theta) \odot (\mathbf{1}_{n\times d}-M)\| + \lambda \|\Theta\|_\star,$$

avec $\|.\|$ la norme nucléaire.

L'article associé à cette méthode est disponible [ici](https://arxiv.org/pdf/1410.2596). L'algorithme est implémenté en langage `R` dans le package `softImpute`.

### Question 3a

Quels sont les hyperparamètres ?

### Solution

L'hyperparamètre est $\lambda$.

### Question 3b

Implémentez cette méthode en reprenant cette base de code. Vous utiliserez l'imputation par zéro codée en question 1a.

In [ ]:
def softimpute(xmiss, lamb, maxit = 1000):

    mask = np.isnan(xmiss)

    ximp = ...

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_thresh = ...
        r = ...
        d_thresh = d_thresh[:r]
        U_thresh = U[:, :r]
        V_thresh = V[:r, :]
        D_thresh = np.diag(d_r)
        svd_r = np.dot(U_thresh, np.dot(D_thresh, V_thresh))
        ximp[mask] = svd_r[mask]

    return ximp

### Solution

In [ ]:
def softimpute(xmiss, lamb, maxit = 1000):

    mask = np.isnan(xmiss)

    ximp = init_mean_imputation(xmiss, mask)

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_thresh = np.maximum(d - lamb, 0)
        r = (d_thresh > 0).sum()
        d_thresh = d_thresh[:r]
        U_thresh = U[:, :r]
        V_thresh = V[:r, :]
        D_thresh = np.diag(d_thresh)
        svd_r = np.dot(U_thresh, np.dot(D_thresh, V_thresh))
        ximp[mask] = svd_r[mask]

    return ximp

In [ ]:
ximp_soft = softimpute(xmiss, 100, maxit = 1000)

In [ ]:
mse(ximp_soft,xfull)

### Question 3c

Pour éviter de faire des itérations non nécessaires, on peut utiliser le critère de convergence implémenté ci-dessous, qui compare l'écart entre deux itérés. Si l'écart est plus faible qu'un seuil `conv_thresh`(fixé à $10^{-5}$ par défaut), l'algorithme a convergé.

Incorporez ce critère de convergence dans la fonction `softimpute`.

In [ ]:
def converged(x_t, x_tplus1, mask, conv_thresh):

    x_t_na = x_t[mask]
    x_tplus1_na = x_tplus1[mask]
    rmse = np.sqrt(np.sum((x_t_na - x_tplus1_na) ** 2))
    denom = np.sqrt((x_t_na ** 2).sum())

    return (rmse / denom) < conv_thresh

### Solution

In [ ]:
def softimpute(xmiss, lamb, maxit = 1000, conv_thresh = 1e-5):

    mask = np.isnan(xmiss)

    ximp = init_mean_imputation(xmiss, mask)

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_thresh = np.maximum(d - lamb, 0)
        r = (d_thresh > 0).sum()
        d_thresh = d_thresh[:r]
        U_thresh = U[:, :r]
        V_thresh = V[:r, :]
        D_thresh = np.diag(d_thresh)
        svd_r = np.dot(U_thresh, np.dot(D_thresh, V_thresh))
        if converged(ximp, svd_r, mask, conv_thresh):
            break
        ximp[mask] = svd_r[mask]

    return ximp

### Question 3d

Utilisez la fonction `choose_lambda` ci-dessous, qui parcourt une grille de valeurs pour $\lambda$ et utilise la fonction `additional_na` définie en question 1d.

`grid_len` permet de fixer la taille de la grille des valeurs.
La grille de valeurs est définie dans la fonction `choose_lambda`, en calculant la valeur singulière maximale du jeu de données imputé par 0.

In [ ]:
def choose_lambda(xmiss, grid_len = 15, maxit = 1000, conv_thresh = 1e-5):

    mask = np.isnan(xmiss)
    ximp_0 = init_zero_imputation(xmiss, mask)

    # generate grid for lambda values
    d = np.linalg.svd(ximp_0, compute_uv=False, full_matrices=False) # svd on imputed dataset with 0
    lambda_max = np.max(d)
    lambda_min = 0.001 * lambda_max
    grid_lambda = np.exp(np.linspace(np.log(lambda_min), np.log(lambda_max), grid_len).tolist())

    cv_error = []
    for lamb in grid_lambda:
        xmiss_addna, mask_maskna = additional_na(xmiss, mask)
        ximp_soft = softimpute(xmiss_addna, lamb, maxit, conv_thresh)
        cv_error.append(np.sqrt(np.nanmean((ximp_soft.flatten() - xmiss.flatten()) ** 2)))

    return cv_error, grid_lambda


### Solution

In [ ]:
res_lambda, grid_lambda = choose_lambda(xmiss, grid_len = 15)
lambda_opt = grid_lambda[np.argmin(res_lambda)]
ximp_soft = softimpute(xmiss, lambda_opt)
mse_soft = mse(ximp_soft, xfull)
print("MSE softImpute:", mse_soft)

# Exercice 4: méthode utilisant l'apprentissage profond

Dans cet exercice, vous allez prédire les valeurs manquantes du jeu de données *Breast Cancer Wisconsin* (de l'exercice 3) en utilisant la méthode `MIWAE`. L'article de recherche qui propose cette méthode est disponible [ici](https://arxiv.org/pdf/1812.02633). Des notebooks sont disponibles sur le répertoire Github [ici](https://github.com/pamattei/miwae/tree/master). Le code est très largement repris de [ce notebook](https://github.com/pamattei/miwae/blob/master/Tensorflow%202%20notebooks/MIWAE_UCI_single_multiple-imputation.ipynb).

Il n'y a pas de questions d'implémentation dans cet exercice, le but est de mieux appréhender la méthode. Les calculs sont possibles sur CPU ou GPU mais seront plus rapides si un GPU est disponible.



`MIWAE` utilise un modèle profond à variables latentes. Soit $Z \in \mathbb{R}^{d_{\textrm{lat}}}$ les variables latentes. L'idée va être d'encoder l'information contenue dans les données dans l'espace latent (de plus petite dimension, $d_{\textrm{lat}} < d$). L'information est ensuite décodée pour revenir dans l'espace des données, avec une distribution $p_\theta(X|Z)$ paramétrée par $f_\theta(Z)$, entrainé par un réseau de neurones.

`MIWAE` propose d'estimer le paramètre $\theta$ en calculant une borne $L_K$ de la vraisemblance observée, telle que  $L_K \leq L_{{\mathrm{{obs}}}}$. Cette borne tire des échantillons pour les variables latentes selon une distribution $p_\gamma(Z|X_{\mathrm{obs}(M)})$, qui encode les données dans l'espace latent, et qui est paramétré par $g_\gamma(X_{\mathrm{obs}(M)})$, entrainé par un autre réseau de neurones.
$$L_K(\theta,\gamma)=\sum_{i=1}^n \mathbb{E}_{Z_{i1},\dots,Z_{iK}\sim p_\gamma(Z|X_{\mathrm{obs}(M)})}\left[\log \frac{1}{K}\sum_{k=1}^K \frac{p_\theta(X_{\mathrm{obs}(M)}|Z_{ik})p(Z_{ik})}{{p_\gamma(Z_{ik}|X_{\mathrm{obs}(M)})}}\right].$$

Finalement, le modèle génératif est le suivant:
$$
\left\{
    \begin{array}{ll}
        Z\sim p_\gamma(Z|X_{\mathrm{obs}(M)};{g_\gamma(X_{\mathrm{obs}(M)})}) &  \textrm{(encodeur)} \\
        X_{\mathrm{obs}(M)} \sim p_\theta(X_{\mathrm{obs}(M)}|Z;{f_\theta(Z)}) & \textrm{(décodeur)}
    \end{array}
\right.
$$

avec $g_\gamma(X_{\mathrm{obs}(M)})$ et $f_\theta(Z)$ entrainés par des réseaux de neurones.

## Question 1: hyperparamètres et modélisation

Quels sont les hyperparamètres ? Quels sont les choix à réaliser pour cette méthode d'imputation ?

### Solution

Les hyperparamètres sont $d_{\textrm{lat}}$ (la dimension de l'espace latent) et $K$ (le nombre d'échantillons pour l'échantillonnage préférentiel).

Les distributions $p(Z)$, $p_\gamma(Z|X_{\mathrm{obs}(M)})$ et $p_\theta(X_{\mathrm{obs}(M)}|Z)$ peuvent être des Gaussiennes, mais un choix plus pertinent peut être fait en fonction des cas et également en fonction du type des variables (quantitatives, catégorielles).

Enfin, on doit choisir l'architecture du réseau de neurones, avec des paramètres associés, notamment :
* le nombre de couches cachés,
* la dimension des couches cachées dans les réseaux de neurones.

Dans la suite, pour les variables latentes, on va choisir une distribution gausienne standard (cette distribution a priori convient la plupart du temps) : $p(Z)=N(0_d,I_{d\times d})$. Pour $p_\gamma(Z|X_{\mathrm{obs}(M)})$, on choisit également une distribution Gaussienne avec une moyenne et une matrice de covariances diagonale. Pour $p_\theta(X_{\mathrm{obs}(M)}|Z)$, on prendra une distribution de Student.

Les réseaux de neurones pour l'encodeur et le décodeur ont la même architecture, avec 3 couches. Par exemple, pour $p_\gamma(Z|X_{\mathrm{obs}(M)})$, on suppose:
$f_\theta(Z)=\sigma(W_1\sigma(W_0 Z + b_0)+ b_1)$, où $\sigma$ est la fonction d'activation ReLU. Les paramètres de la loi de Student sont ensuite estimés avec :
\begin{align*}
\mu_\theta&=W_\mu f_\theta(Z) + b_\mu \quad \textrm{(moyenne)} \\
\Sigma_\theta&=\textrm{Softplus}(\textrm{Diag}(W_\sigma f_\theta(Z) + b_\sigma))+10^{-3} \quad \textrm{(matrice de covariances)} \\
\nu_\theta &= \textrm{Softplus}(W_\nu f_\theta(Z) + b_\nu)+3 \quad \textrm{(nombre de degrés de liberté)},
\end{align*}
On rajoute $10^{-3}$ pour effectivement obtenir une matrice de covariances, et $3$ pour avoir au moins trois degrés de liberté et ne pas avoir des queues trop lourdes. Ces choix sont expliqués plus en détail dans le notebook de la méthode `MIWAE` cité ci-dessus.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # determine si CUDA est disponible (utilisation GPU)

In [ ]:
h = 128 # number of hidden units in (same for all MLPs)
d_lat = 10 # dimension of the latent space
K = 20 # number of IS during training

In [ ]:
p_z = td.Independent(td.Normal(loc=torch.zeros(d_lat).to(device), scale=torch.ones(d_lat).to(device)), 1)

In [ ]:
decoder = nn.Sequential(
    torch.nn.Linear(d_lat, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 3 * d),  # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
)

In [ ]:
encoder = nn.Sequential(
    torch.nn.Linear(d, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 2 * d_lat),  # the encoder will output both the mean and the diagonal covariance
)

In [ ]:
def weights_init(layer): # this function will be used for initializing the neural networks
  if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)

Ci-dessous, voici la fonction qui permet de calculer la borne $L_K$ à maximiser.

In [ ]:
def miwae_loss(iota_x, mask):
  batch_size = iota_x.shape[0]
  out_encoder = encoder(iota_x)
  q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d_lat], scale=torch.nn.Softplus()(out_encoder[..., d_lat:(2*d_lat)])), 1)

  zgivenx = q_zgivenxobs.rsample([K])
  zgivenx_flat = zgivenx.reshape([K*batch_size, d_lat])

  out_decoder = decoder(zgivenx_flat)
  all_means_obs_model = out_decoder[..., :d]
  all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., d:(2*d)]) + 0.001
  all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*d):(3*d)]) + 3

  data_flat = torch.Tensor.repeat(iota_x,[K, 1]).reshape([-1, 1])
  tiledmask = torch.Tensor.repeat(mask,[K, 1])

  all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1, 1]), scale=all_scales_obs_model.reshape([-1, 1]), df=all_degfreedom_obs_model.reshape([-1, 1])).log_prob(data_flat)
  all_log_pxgivenz = all_log_pxgivenz_flat.reshape([K * batch_size, d])

  logpxobsgivenz = torch.sum(all_log_pxgivenz * (1 - tiledmask), 1).reshape([K, batch_size])
  logpz = p_z.log_prob(zgivenx)
  logq = q_zgivenxobs.log_prob(zgivenx)

  neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq,0))

  return neg_bound

## Question 2: standardisation

Comme beaucoup de méthodes d'apprentissage profond, il faut standardiser les données.

On standardise le jeu de données qui contient des valeurs manquantes avec le code suivant. Quel peut être le problème ici ?

In [ ]:
mean_xmiss = np.nanmean(xmiss, 0)
std_xmiss = np.nanstd(xmiss, 0)
xmiss = (xmiss - mean_xmiss) / std_xmiss

ximp_0 = init_zero_imputation(xmiss, mask)

### Solution

On standardise les données en calculant moyenne et un écart-type sur les données imputées par 0. Ces quantités peuvent donc être biaisées, mais on peut faire l'hypothèse que cela n'aura pas de répercussion à la fin de l'entrainement du modèle.

## Question 3: entraînement du modèle

Avec quel algorithme optimiser la borne $L_K$ ?

### Solution

On utilise l'algorithme de descente de gradient stochastique (SGD) par lots (batchs), que l'on rappelle ci-dessous en pseudo-code:

`for epoch=0 in NB_epochs:` # boucle sur les époques
*   `Tirer une permutation aléatoire des données et construire les lots de taille n_batch`
*   `Pour chaque lot`: # boucle sur les lots
  + `Mettre à jour les paramètres`
$$ (\theta^{(t+1)},\gamma^{(t+1)})=(\theta^{(t)},\gamma^{(t)})-\lambda \: \partial_{(\theta,\gamma)} L_K(\theta^{(t)},\gamma^{(t)}), \quad \textrm{avec $\lambda$ est le pas du gradient.}$$





L'utilisateur choisit le pas de gradient, le nombre d'époque et la taille des lots.

En pratique, on va préférer l'utilisation de l'algorithme d'optimisation `Adam`, une extension du `SGD` avec un pas de gradient adaptatif et un momentum.

Voici le code pour l'entaînement du modèle.

In [ ]:
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)

In [ ]:
encoder.to(device) # we'll use the GPU or CPU depending on the available device
decoder.to(device)

In [ ]:
miwae_loss_train = np.array([])
mse_train = np.array([])
mse_train2 = np.array([])
bs = 64 # batch size
n_epochs = 2002 # 1 epoch = all the data are used once
ximp_MIWAE = np.copy(ximp_0) # This will be out imputed data matrix

encoder.apply(weights_init)
decoder.apply(weights_init)

for ep in range(1, n_epochs):
  perm = np.random.permutation(n) # We use the "random reshuffling" version of SGD
  batches_data = np.array_split(ximp_0[perm,], n/bs)
  batches_mask = np.array_split(mask[perm,], n/bs)
  for it in range(len(batches_data)):
    optimizer.zero_grad()
    encoder.zero_grad()
    decoder.zero_grad()
    b_data = torch.from_numpy(batches_data[it]).float().to(device)
    b_mask = torch.from_numpy(batches_mask[it]).float().to(device)
    loss = miwae_loss(iota_x=b_data, mask=b_mask)
    loss.backward()
    optimizer.step()
  if ep % 100 == 1:
    print('Epoch %g' %ep)
    print('MIWAE likelihood bound  %g' %(-np.log(K) - miwae_loss(iota_x=torch.from_numpy(ximp_0).float().to(device), mask=torch.from_numpy(mask).float().to(device)).cpu().data.numpy())) # Gradient step

## Question 4: imputation

La dernière étape consiste à prédire les valeurs manquantes. On va réaliser une imputation simple, mais une imputation multiple est également possible (cf [notebook spécifique](https://github.com/pamattei/miwae/blob/master/Tensorflow%202%20notebooks/MIWAE_UCI_single_multiple-imputation.ipynb)).

Quelle quantité calculer pour prédire les valeurs manquantes ?

### Solution

   L'idée de `MIWAE` est de calculer l'espérance conditionnelle de $X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)}$ ci-dessous:

   
   \begin{align*}
        &\mathbb{E}[X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)}] \\
        &=\int X_{\mathrm{mis}(M)} p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)})dX_{\mathrm{mis}(M)} \\
        &= \int \int X_{\mathrm{mis}(M)} p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)},Z)p_\theta(Z|X_{\mathrm{obs}(M)})dZdX_{\mathrm{mis}(M)} \\
        &= \int \int X_{\mathrm{mis}(M)} \frac{p_\theta(Z|X_{\mathrm{obs}(M)})}{{p_\gamma(Z|X_{\mathrm{obs}(M)})}}{p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)},Z)p_\gamma(Z|X_{\mathrm{obs}(M)})}dZdX_{\mathrm{mis}(M)}
    \end{align*}
Elle n'est pas explicite, donc on utilise un échantillonnage préférentiel dit auto-normalisé (plus de détails sur cette méthode d'échantillonnage sont disponible [ici, Section 9.2](https://artowen.su.domains/mc/)):
$$\sum_{l=1}^L w_l X_{\mathrm{mis}(M)}^{(l)}$$
avec $w_l=\frac{r_l}{\sum_{l=1}^L r_l}$, $r_l=\frac{p_\theta(X_{\mathrm{obs}(M)}|Z^{(l)})p(Z^{(l)})}{{p_\gamma(Z^{(l)}|X_{\mathrm{obs}(M)})}}$, et
$(X_{\mathrm{mis}(M)}^{(l)},Z^{(l)}) \overset{\mathrm{iid}}{\sim} {p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)},Z)p_\gamma(Z|X_{\mathrm{obs}(M)})}$

In [ ]:
def miwae_impute(iota_x,mask, L):
  batch_size = iota_x.shape[0]
  out_encoder = encoder(iota_x)
  q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d_lat], scale=torch.nn.Softplus()(out_encoder[..., d_lat:(2*d_lat)])), 1)

  zgivenx = q_zgivenxobs.rsample([L])
  zgivenx_flat = zgivenx.reshape([L * batch_size, d_lat])

  out_decoder = decoder(zgivenx_flat)
  all_means_obs_model = out_decoder[..., :d]
  all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., d:(2*d)]) + 0.001
  all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*d):(3*d)]) + 3

  data_flat = torch.Tensor.repeat(iota_x,[L, 1]).reshape([-1, 1]).to(device)
  tiledmask = torch.Tensor.repeat(mask,[L, 1]).to(device)

  all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1, 1]), scale=all_scales_obs_model.reshape([-1, 1]), df=all_degfreedom_obs_model.reshape([-1, 1])).log_prob(data_flat)
  all_log_pxgivenz = all_log_pxgivenz_flat.reshape([L * batch_size, d])

  logpxobsgivenz = torch.sum(all_log_pxgivenz * (1 - tiledmask), 1).reshape([L, batch_size])
  logpz = p_z.log_prob(zgivenx)
  logq = q_zgivenxobs.log_prob(zgivenx)

  xgivenz = td.Independent(td.StudentT(loc=all_means_obs_model, scale=all_scales_obs_model, df=all_degfreedom_obs_model), 1)

  imp_weights = torch.nn.functional.softmax(logpxobsgivenz + logpz - logq, 0) # these are w_1,....,w_L for all observations in the batch
  xms = xgivenz.sample().reshape([L, batch_size, d])
  xm = torch.einsum('ki,kij->ij', imp_weights, xms)

  return xm


In [ ]:
ximp_MIWAE[mask] = miwae_impute(iota_x=torch.from_numpy(ximp_0).float().to(device), mask=torch.from_numpy(mask).float().to(device), L=10).cpu().data.numpy()[mask]
ximp_MIWAE_destandardized = ximp_MIWAE * std_xmiss + mean_xmiss
mse_MIWAE = mse(ximp_MIWAE_destandardized, xfull)
print("MSE MIWAE:", mse_MIWAE)